In [6]:
import tensorflow as tf
from tensorflow.keras import layers,Model
from tensorflow.keras.activations import tanh
from tensorflow.keras.activations import softmax


In [7]:
class Attention_model(Model):
    def __init__(self, units):
        super(Attention_model, self).__init__()
        self.units=units
        # build your Dense layer
        self.W1 = tf.keras.layers.Dense(units)
        # build your Dense layer
        self.W2 = tf.keras.layers.Dense(units)
        # build your final Dense layer with unit 1
        # self.V = tf.keras.layers.Dense(1, activation='softmax')
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):
        # features shape: (batch_size, 8*8, embedding_dim)
        # hidden shape: (batch_size, hidden_size)
        
        # Expand the hidden shape to shape: (batch_size, 1, hidden_size)
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        # build your score funciton to shape: (batch_size, 8*8, units)
        score = tanh(self.W1(features) + self.W2(hidden_with_time_axis))
        # extract your attention weights with shape: (batch_size, 8*8, 1)
        score = self.V(score)
        attention_weights = softmax(score, axis=1)

        # shape: create the context vector with shape (batch_size, 8*8,embedding_dim)
        context_vector = attention_weights * features
        # reduce the shape to (batch_size, embedding_dim)
        # context_vector = tf.reduce_sum(context_vector, axis=1)
        context_vector = tf.reduce_mean(context_vector, axis=1)

        return context_vector, attention_weights   

In [22]:
units = 32
attention = Attention_model(units)
features = tf.zeros([32, 64, 2048], tf.float32)
hidden = tf.zeros([32, 16], tf.float32)
# output = attention(features,hidden)


In [23]:
# method_list = [func for func in dir(Attention_model) if callable(getattr(Attention_model, func))]
# method_list = [func for func in dir(Attention_model) if callable(getattr(Attention_model, func)) and not func.startswith("__")]


In [24]:
# attention()
input_shape = (None, 32, 32, 3)

model.build((32, 64, 2048),)
attention.compile()
attention.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.